In [1]:
import pandas as pd
import numpy as np
import os
import itertools

os.chdir("../")

In [2]:
### UTILS ###
def remove_cols(df: pd.DataFrame, toremove: list) -> pd.DataFrame:
    """ Removes every column from passed list from dataframe """
    
    if df is None:
        raise ValueError("Passed dataframe is None")
    if toremove is None or toremove == []:
        raise ValueError("Passed list of columns is invalid")
        
    for column in toremove:
        if column is None or column not in df.columns:
            print('Column not found: ', column)
            continue
        df.drop(column, axis=1, inplace=True)
    return df


def drop_duplicates_by(df: pd.DataFrame, columns: list, keep: str = "first") -> pd.DataFrame:
    """ Removes duplicate rows based on one or more columns """
    
    if columns == []:
        columns = df.columns
    if df is None or not all(col in df.columns for col in columns):
        raise ValueError("One or more specified columns do not exist in the dataframe")

    cleaned_df = df.drop_duplicates(subset=columns, keep=keep)
    return cleaned_df.reset_index(drop=True)


def drop_na_rows(df: pd.DataFrame, columns: list = None) -> pd.DataFrame:
    """ Remove rows with missing NA values in specified columns or in the entire dataframe """

    if df is None:
        raise ValueError("Passed dataframe is None")
        
    cleaned_df = df.dropna(subset=columns) if columns else df.dropna()
    return cleaned_df.reset_index(drop=True)


def split_date_column(df: pd.DataFrame, date_col: str) -> pd.DataFrame:
    """ Split a date column (format YYYY-MM-DD) into numeric year/day and abbreviated month (Jan, Feb, Mar, ...) """
    
    if df is None:
        raise ValueError("Passed dataframe is None")
    if date_col not in df.columns:
        raise ValueError(f"Column '{date_col}' not found in dataframe.")
    
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    
    df["year"] = df[date_col].dt.year.astype(int)
    df["month"] = df[date_col].dt.strftime("%b")  
    df["day"] = df[date_col].dt.day.astype(int)
    
    return df


def monthly_average_temperature(df: pd.DataFrame, month: str, year: int) -> float:
    """ Calculate the average temperature for a given month and year, based on the max and min temps for each day """

    required_cols = {"MinTemp", "MaxTemp", "month", "year"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"DataFrame must contain columns: {required_cols}")

    df_filtered = df[(df["month"] == month) & (df["year"] == year)]

    if df_filtered.empty:
        raise ValueError(f"No temperature data found for {month} {year}")

    df_filtered["daily_mean"] = (df_filtered["MaxTemp"] + df_filtered["MinTemp"]) / 2
    monthly_mean = df_filtered["daily_mean"].mean()

    return round(monthly_mean, 2)


def max_monthly_value(df: pd.DataFrame, month: str, year: int, column: str) -> float:
    """ Return the maximum value recorded for a column in a given month and year """
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    
    df_filtered = df[(df["month"] == month) & (df["year"] == year)]
    if df_filtered.empty:
        raise ValueError(f"No data found for {month} {year}.")
    
    return df_filtered[column].max(skipna=True)


def min_monthly_value(df: pd.DataFrame, month: str, year: int, column: str) -> float:
    """ Return the minimum value recorded for a column in a given month and year """
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    
    df_filtered = df[(df["month"] == month) & (df["year"] == year)]
    if df_filtered.empty:
        raise ValueError(f"No data found for {month} {year}.")
    
    return df_filtered[column].min(skipna=True)


def avg_monthly_value(df: pd.DataFrame, month: str, year: int, column: str) -> float:
    """ Return the average value recorded for a column in a given month and year """
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    
    df_filtered = df[(df["month"] == month) & (df["year"] == year)]
    if df_filtered.empty:
        raise ValueError(f"No data found for {month} {year}.")
    
    monthly_avg = df_filtered[column].mean(skipna=True)
    
    return round(monthly_avg, 2)

In [3]:
# load dataset
df = pd.read_csv('data/weatherAUS.csv', encoding='utf-8')

# display basic info
print('dataset basic info:\n')
df.info()
print('\n')
display(df.head())

dataset basic info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [4]:
"""
    En primer lugar reduciremos el dataset a solo las observaciones correspondientes a la ciudad de Sydney,
    ya que como nuestro objetivo es unicamente la visualización de datos, no necesitamos todos los registros.
"""

df = df[df["Location"].str.lower() == "sydney"]

# df cleaning
df = remove_cols(df, ['RainToday', 'RainTomorrow'])
df = drop_duplicates_by(df, ['Date'])
df = drop_na_rows(df)
df = split_date_column(df, 'Date')

In [5]:
display(df.describe(include='all').T)
df.head()

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
Date,1693,NaN,NaN,NaN,2014-12-15 03:24:08.080330752,2010-10-20 00:00:00,2013-10-12 00:00:00,2014-12-30 00:00:00,2016-03-27 00:00:00,2017-06-25 00:00:00,NaN
Location,1693,1,Sydney,1693,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MinTemp,1693.0,NaN,NaN,NaN,15.03733,5.0,11.3,15.1,18.9,27.1,4.522421
MaxTemp,1693.0,NaN,NaN,NaN,23.454637,11.7,20.2,23.3,26.4,45.8,4.477532
Rainfall,1693.0,NaN,NaN,NaN,2.827525,0.0,0.0,0.0,1.0,94.4,8.221317
Evaporation,1693.0,NaN,NaN,NaN,5.394152,0.0,3.2,5.0,7.4,18.4,2.854083
Sunshine,1693.0,NaN,NaN,NaN,7.400354,0.0,4.6,8.4,10.3,13.6,3.746518
WindGustDir,1693,16,W,321,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WindGustSpeed,1693.0,NaN,NaN,NaN,41.580035,17.0,31.0,41.0,48.0,96.0,12.698272
WindDir9am,1693,16,W,591,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,year,month,day
0,2010-10-20,Sydney,12.9,20.3,0.2,3.0,10.9,ENE,37.0,W,...,57.0,1028.8,1025.6,3.0,1.0,16.9,19.8,2010,Oct,20
1,2010-10-21,Sydney,13.3,21.5,0.0,6.6,11.0,ENE,41.0,W,...,58.0,1025.9,1022.4,2.0,5.0,17.6,21.3,2010,Oct,21
2,2010-10-22,Sydney,15.3,23.0,0.0,5.6,11.0,NNE,41.0,W,...,63.0,1021.4,1017.8,1.0,4.0,19.0,22.2,2010,Oct,22
3,2010-10-26,Sydney,12.9,26.7,0.2,3.8,12.1,NE,33.0,W,...,56.0,1018.0,1015.0,1.0,5.0,17.8,22.5,2010,Oct,26
4,2010-10-27,Sydney,14.8,23.8,0.0,6.8,9.6,SSE,54.0,SSE,...,69.0,1016.0,1014.7,2.0,7.0,20.2,20.6,2010,Oct,27


In [6]:
df.to_csv("data/weatherAUS_clean.csv", index=False, encoding="utf-8")

In [7]:
"""
    Crearé otro dataset en el que los registros sean mensuales en lugar de diarios (promedios)
"""

def build_monthly_summary(df: pd.DataFrame) -> pd.DataFrame:
    """Create monthly dataset from daily registers"""
    
    required_cols = ["year", "month", "MinTemp", "MaxTemp", "Rainfall", 
                     "WindSpeed9am", "WindSpeed3pm", "Sunshine", "Evaporation"]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Missing column '{col}' in DataFrame")

    df["year"] = df["year"].astype(int)
    df["month"] = df["month"].astype(str).str.title().str[:3]

    df["TempMean"] = (df["MaxTemp"] + df["MinTemp"]) / 2
    df["WindSpeedAvg"] = (df["WindSpeed9am"] + df["WindSpeed3pm"]) / 2

    monthly_summary = (
        df.groupby(["year", "month"], as_index=False)
        .agg(
            avg_temp=("TempMean", "mean"),
            max_temp=("MaxTemp", "max"),
            min_temp=("MinTemp", "min"),
            avg_rainfall=("Rainfall", "mean"),
            avg_windspeed=("WindSpeedAvg", "mean"),
            avg_sunshine=("Sunshine", "mean"),
            avg_evaporation=("Evaporation", "mean"),
        )
    )

    month_order = ["Jan","Feb","Mar","Apr","May","Jun",
                   "Jul","Aug","Sep","Oct","Nov","Dec"]
    monthly_summary["month"] = pd.Categorical(monthly_summary["month"], categories=month_order, ordered=True)
    monthly_summary = monthly_summary.sort_values(["year", "month"]).reset_index(drop=True)
    return monthly_summary


def build_monthly_summary_filled(df: pd.DataFrame) -> pd.DataFrame:
    """ Create monthly dataset and clean missing registers """
    
    base = build_monthly_summary(df)
    base = base[~base["year"].isin([2010, 2012, 2017])]

    all_years = sorted(base["year"].unique())
    month_order = ["Jan","Feb","Mar","Apr","May","Jun",
                   "Jul","Aug","Sep","Oct","Nov","Dec"]
    all_combinations = pd.DataFrame(itertools.product(all_years, month_order), columns=["year", "month"])

    merged = pd.merge(all_combinations, base, on=["year", "month"], how="left")
    merged["month_num"] = merged["month"].apply(lambda m: month_order.index(m) + 1)

    numeric_cols = ["avg_temp", "max_temp", "min_temp", 
                    "avg_rainfall", "avg_windspeed", 
                    "avg_sunshine", "avg_evaporation"]

    filled = []
    for year in all_years:
        subset = merged[merged["year"] == year].copy()
        subset = subset.sort_values("month_num")

        subset[numeric_cols] = subset[numeric_cols].interpolate(method="linear", limit_direction="both")
        filled.append(subset)

    filled_df = pd.concat(filled, ignore_index=True)
    filled_df[numeric_cols] = filled_df[numeric_cols].round(2)

    filled_df = filled_df.sort_values(["year", "month"]).reset_index(drop=True)
    return filled_df

monthly_df = build_monthly_summary_filled(df)
print(monthly_df.head())


   year month  avg_temp  max_temp  min_temp  avg_rainfall  avg_windspeed  \
0  2013   Apr     19.58     28.70     10.90          5.73          15.43   
1  2013   Aug     15.77     24.70      7.00          0.49          18.27   
2  2013   Dec     22.47     36.00     12.70          1.00          18.24   
3  2013   Feb     23.36     39.65     15.35          1.80          17.67   
4  2013   Jan     24.10     45.80     16.80          1.42          16.77   

   avg_sunshine  avg_evaporation  month_num  
0          7.75             3.97          4  
1          8.94             4.47          8  
2          9.37             7.83         12  
3          7.54             7.14          2  
4          7.38             7.89          1  


In [8]:
"""
    Haré algunos cambios en los datos, solo para las visualizaciones, ya que no se busca analizar los datos reales.
"""
monthly_df.loc[monthly_df['year'] == 2013, 'avg_temp'] *= 0.98
monthly_df.loc[monthly_df['year'] == 2015, 'avg_temp'] *= 1.05
monthly_df.loc[monthly_df['year'] == 2016, 'avg_temp'] *= 1.1

monthly_df.loc[monthly_df['year'] == 2013, 'avg_sunshine'] *= 0.8
monthly_df.loc[monthly_df['year'] == 2014, 'avg_sunshine'] *= 0.9
monthly_df.loc[monthly_df['year'] == 2015, 'avg_sunshine'] *= 1.05
monthly_df.loc[monthly_df['year'] == 2016, 'avg_sunshine'] *= 1.15

def get_season(month):
    if month in ["Dec", "Jan", "Feb"]:
        return "Summer"
    elif month in ["Mar", "Apr", "May"]:
        return "Autumn"
    elif month in ["Jun", "Jul", "Aug"]:
        return "Winter"
    else:
        return "Spring"

monthly_df["season"] = monthly_df["month"].apply(get_season)

adjustments = {
    (2015, "Summer"): 1.5,
    (2015, "Autumn"): 0.9,
    (2016, "Winter"): 0.88,
    (2016, "Spring"): 1.05,
    (2016, "Summer"): 1.2,
    (2014, "Winter"): 0.9,
    (2015, "Winter"): 0.8,
    (2013, "Spring"): 1.05
}

for (yr, season), factor in adjustments.items():
    monthly_df.loc[
        (monthly_df["season"] == season) & (monthly_df["season"] == season),
        "avg_sunshine"
    ] *= factor

monthly_df.loc[monthly_df['season'] == 'winter', 'avg_sunshine'] *= 0.9
monthly_df.loc[monthly_df['season'] == 'winter', 'avg_sunshine'] *= 0.7
monthly_df.loc[monthly_df['season'] == 'summer', 'avg_sunshine'] *= 1.1
    
monthly_df['avg_temp'] = monthly_df['avg_temp'].round(2)
monthly_df['avg_sunshine'] = monthly_df['avg_sunshine'].round(2)
print(monthly_df.head())

   year month  avg_temp  max_temp  min_temp  avg_rainfall  avg_windspeed  \
0  2013   Apr     19.19     28.70     10.90          5.73          15.43   
1  2013   Aug     15.45     24.70      7.00          0.49          18.27   
2  2013   Dec     22.02     36.00     12.70          1.00          18.24   
3  2013   Feb     22.89     39.65     15.35          1.80          17.67   
4  2013   Jan     23.62     45.80     16.80          1.42          16.77   

   avg_sunshine  avg_evaporation  month_num  season  
0          5.58             3.97          4  Autumn  
1          4.53             4.47          8  Winter  
2         13.49             7.83         12  Summer  
3         10.86             7.14          2  Summer  
4         10.63             7.89          1  Summer  


In [9]:
monthly_df.to_csv("data/weatherAUS_clean.csv", index=False, encoding="utf-8")